In [1]:
!python --version

Python 3.5.2 :: Anaconda custom (x86_64)


# CLIXO Ontology Tree Generator

This is a notebook to generate tree data file from original table and annotations.


In [5]:
import pandas as pd
treeSourceUrl = './preds_yeastnet_no_gi_0.04_0.5.txt.propagate.small_parent_tree'

# Load the tree data
treeColNames = ['parent', 'child', 'type', 'in_tree']
tree = pd.read_csv(treeSourceUrl, delimiter='\t', names=treeColNames)
tree.tail(10)

,parent,child,type,in_tree
136991,10702,YHR083W,gene,NOT_TREE
136992,10701,YHR083W,gene,NOT_TREE
136993,10700,YHR083W,gene,NOT_TREE
136994,9450,YHR083W,gene,NOT_TREE
136995,10706,YHR083W,gene,NOT_TREE
136996,10051,YHR083W,gene,NOT_TREE
136997,10704,YHR083W,gene,NOT_TREE
136998,10699,YHR083W,gene,NOT_TREE
136999,10705,YHR083W,gene,NOT_TREE
137000,9816,YHR083W,gene,NOT_TREE


## Build Base CyJS Network

In [12]:
import json

clixoTree = {
    'data': {
        'name': 'CLIXO Tree'
    },
    'elements': {
        'nodes': [],
        'edges': []
    }
}

print(json.dumps(clixoTree, indent=4))

{
    "data": {
        "name": "CLIXO Tree"
    },
    "elements": {
        "edges": [],
        "nodes": []
    }
}


In [13]:
def get_node(id):
    node = {
        'data': {
            'id': id
        }
    }
        
    return node

def get_edge(source, target, itr, is_tree):
    edge = {
        'data': {
            'source': target,
            'target': source,
            'interaction': itr,
            'is_tree': is_tree
        }
    }
    
    return edge

In [14]:
node_set= set()

edges = []
PREFIX = 'CLIXO:'

for row in tree.itertuples():
    source = PREFIX + str(row[1])
    child = PREFIX + str(row[2])
    node_set.add(source)
    node_set.add(child)
    edges.append(get_edge(source, child, row[3], row[4]))

In [15]:
nodes = []

for id in node_set:
    node = get_node(id)
    nodes.append(node)

In [16]:
clixoTree['elements']['nodes'] = nodes
clixoTree['elements']['edges'] = edges

with open('./data/clixo-tree.cyjs', 'w') as outfile:
    json.dump(clixoTree, outfile)

## Layout by library

In [ ]:
import networkx as nx

DG=nx.DiGraph()

for node in nodes:
    DG.add_node(node['data']['id'])

for edge in edges:
    DG.add_edge(edge['data']['source'], edge['data']['target'])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
nx.draw_circular(DG)

In [ ]:
pos = nx.nx_pydot.pydot_layout(DG)